In [5]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 💥 AD-AS Shocks and Dynamic Adjustment

This simulation demonstrates how a standard macroeconomic model (featuring an IS curve, a Phillips curve, and a Taylor rule) responds dynamically to different types of temporary economic shocks.

We can observe how output ($Y$) and inflation ($\pi$) deviate from their long-run equilibrium values ($\bar{Y}$, $\pi^*$) following a shock and how the interaction of aggregate demand, aggregate supply, and monetary policy guides the economy back towards equilibrium over time.

# 🧠 Core Equations

The model consists of three key equations:

1.  **IS Curve (Aggregate Demand):** Output depends negatively on the real interest rate gap, plus any demand shocks ($\varepsilon^{AD}$).
    $$Y_t = \bar{Y} - \alpha (r_t - \bar{r}) + \varepsilon^{AD}_t$$

2.  **AS Curve (Phillips Curve):** Inflation depends on expected inflation ($\pi^e$), the output gap, and any supply shocks ($\varepsilon^{AS}$).
    $$\pi_t = \pi^e + \lambda (Y_t - \bar{Y}) + \varepsilon^{AS}_t$$
    *(Note: In this simulation, $\pi^e$ is assumed constant unless hit by an 'Expectations' shock initially).*

3.  **Monetary Policy Rule (Taylor Rule):** The central bank sets the real interest rate based on the natural rate ($\bar{r}$) and the deviation of *past* inflation from the target ($\pi^*$).
    $$r_t = \bar{r} + \phi_\pi (\pi_{t-1} - \pi^*)$$

We simulate this system period by period following a one-time shock introduced early in the simulation.

# ⚡ Types of Shocks Simulated

This simulation allows introducing a temporary (one-period) shock in period `t=2`. The types are:

* **None:** Baseline simulation with no shock. The economy should stay at or quickly converge to equilibrium ($\bar{Y}$, $\pi^*$) if started there ($\pi^e = \pi^*$).
* **Negative Demand Shock ($\varepsilon^{AD} < 0$):** Represents a sudden decrease in autonomous demand (e.g., drop in consumer confidence, fall in government spending).
    * *Expected Impact:* Shifts AD left. Initially lowers output ($Y$) and puts downward pressure on inflation ($\pi$). Policy rate ($r$) likely falls as inflation drops.
* **Positive Supply Shock ($\varepsilon^{AS} < 0$):** Represents a sudden decrease in production costs or a temporary boost in productivity (e.g., fall in oil prices).
    * *Expected Impact:* Shifts AS right/down. Initially lowers inflation ($\pi$) and raises output ($Y$). Policy rate ($r$) likely falls.
* **Negative Supply Shock ($\varepsilon^{AS} > 0$):** Represents a sudden increase in production costs (e.g., rise in oil prices). Often called "stagflationary".
    * *Expected Impact:* Shifts AS left/up. Initially raises inflation ($\pi$) and lowers output ($Y$). Creates a policy dilemma, as fighting inflation requires raising $r$ (further lowering $Y$), while supporting $Y$ requires lowering $r$ (further raising $\pi$).
* **Expectations Shock ($\pi_0$ increases):** Represents a sudden jump in initial inflation expectations (or actual initial inflation), perhaps due to news or policy announcements. $\pi^e$ itself remains constant later.
    * *Expected Impact:* Starts inflation higher ($\pi_0$). Policy rate ($r$) immediately increases. Higher $r$ reduces output ($Y$). Inflation gradually comes down as policy works and the output gap potentially turns negative.

Select a shock type and magnitude using the widgets to observe these different dynamic responses.

In [6]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown # Use IntSlider for T, Dropdown for shock type
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def ad_as_shock_sim(pi_e=2.0, pi_star=2.0, Y_bar=100, r_bar=2.0,
                    phi_pi=1.5, lamb_as=0.3, alpha_is=5.0,
                    shock_type='None', shock_mag=5.0, T=25):
    """
    Simulates the dynamic response of an AD-AS model to various one-period shocks.

    Args:
        pi_e (float): Expected inflation (pi^e). Assumed constant after t=0.
        pi_star (float): Target inflation rate (pi*).
        Y_bar (float): Potential Output (Y_bar).
        r_bar (float): Natural real rate of interest (r_bar).
        phi_pi (float): Responsiveness of policy rate to inflation gap in Taylor rule.
        lamb_as (float): Slope of the Phillips curve / AS curve (lambda).
        alpha_is (float): Sensitivity of output to interest rate gap in IS curve (alpha).
        shock_type (str): Type of shock ('None', 'Negative Demand', 'Positive Supply',
                          'Negative Supply', 'Expectations').
        shock_mag (float): Magnitude of the shock (absolute value).
        T (int): Time Horizon (number of periods).
    """
    # Ensure T is an integer
    T = int(T)
    if T < 5: T = 5 # Need enough periods to see dynamics

    # Input validation for parameters
    alpha_is = max(alpha_is, 1e-6)
    phi_pi = max(phi_pi, 0)
    lamb_as = max(lamb_as, 1e-6)
    shock_mag = abs(shock_mag) # Ensure magnitude is positive

    # Arrays to store results
    pi = np.zeros(T) # Inflation rate
    Y = np.zeros(T)  # Output
    r = np.zeros(T)  # Real interest rate set by policy
    eps_AD = np.zeros(T) # AD shock series
    eps_AS = np.zeros(T) # AS shock series

    # --- Initial Period (t=0) ---
    pi[0] = pi_e # Start inflation at expected level
    # Apply initial expectations shock if selected
    if shock_type == 'Expectations':
        pi[0] += shock_mag # Increase initial inflation

    # Calculate initial interest rate based on initial inflation deviation
    r[0] = r_bar + phi_pi * (pi[0] - pi_star)
    # Calculate initial output based on initial interest rate (no AD shock at t=0)
    Y[0] = Y_bar - alpha_is * (r[0] - r_bar)

    # --- Introduce Shocks in Period t=2 ---
    shock_period = 2
    if T > shock_period: # Only apply shock if simulation is long enough
        if shock_type == "Negative Demand":
            eps_AD[shock_period] = -shock_mag
        elif shock_type == "Positive Supply":
            eps_AS[shock_period] = -shock_mag # Negative shock reduces inflation
        elif shock_type == "Negative Supply":
            eps_AS[shock_period] = shock_mag  # Positive shock increases inflation

    # --- Simulation Loop (t=1 to T-1) ---
    for t in range(1, T):
        # 1. Taylor Rule: Set interest rate based on *previous* inflation
        r[t] = r_bar + phi_pi * (pi[t-1] - pi_star)
        # Optional: Add bounds to interest rate if needed
        # r[t] = max(r[t], 0.0)

        # 2. IS Curve: Determine output based on *current* interest rate and AD shock
        Y[t] = Y_bar - alpha_is * (r[t] - r_bar) + eps_AD[t]

        # 3. AS Curve (Phillips Curve): Determine inflation based on *current* output gap and AS shock
        # Expected inflation pi_e is constant after t=0
        pi[t] = pi_e + lamb_as * (Y[t] - Y_bar) + eps_AS[t]
        # Optional: Add bounds to inflation if needed
        # pi[t] = max(pi[t], -5.0)

    # --- Plotting ---
    fig, axes = plt.subplots(2, 2, figsize=(14, 10)) # 2x2 grid
    axes = axes.ravel() # Flatten for easy indexing

    # Calculate dynamic plot limits (find overall min/max across relevant variables)
    y_min_data = min(Y.min(), Y_bar)
    y_max_data = max(Y.max(), Y_bar)
    y_range = max(y_max_data - y_min_data, 1)
    y_lim_lower = y_min_data - y_range*0.1 - 1
    y_lim_upper = y_max_data + y_range*0.1 + 1

    pi_min_data = min(pi.min(), pi_star, pi_e)
    pi_max_data = max(pi.max(), pi_star, pi_e)
    pi_range = max(pi_max_data - pi_min_data, 0.1)
    pi_lim_lower = pi_min_data - pi_range*0.1 - 0.5
    pi_lim_upper = pi_max_data + pi_range*0.1 + 0.5

    r_min_data = min(r.min(), r_bar)
    r_max_data = max(r.max(), r_bar)
    r_range = max(r_max_data - r_min_data, 0.1)
    r_lim_lower = r_min_data - r_range*0.1 - 0.5
    r_lim_upper = r_max_data + r_range*0.1 + 0.5

    time_periods = np.arange(T)

    # Plot 0: Output Over Time
    axes[0].plot(time_periods, Y, marker='o', linestyle='-', label='Output Y(t)', color='blue', markersize=4)
    axes[0].axhline(Y_bar, linestyle='--', color='gray', label=f'Potential Y_bar = {Y_bar:.1f}')
    if shock_type != 'None' and T > shock_period:
        axes[0].axvline(shock_period, color='red', linestyle=':', alpha=0.7, label=f'Shock at t={shock_period}')
    axes[0].set_title("Output (Y) Over Time")
    axes[0].set_xlabel("Time Period (t)")
    axes[0].set_ylabel("Output (Y)")
    axes[0].grid(True, alpha=0.6)
    axes[0].legend(fontsize='small')
    axes[0].set_ylim(y_lim_lower, y_lim_upper)

    # Plot 1: Inflation Over Time
    axes[1].plot(time_periods, pi, marker='o', linestyle='-', color='crimson', label='Inflation pi(t)', markersize=4)
    axes[1].axhline(pi_star, linestyle='--', color='gray', label=f'Target pi* = {pi_star:.1f}%')
    axes[1].axhline(pi_e, linestyle=':', color='lightcoral', label=f'Expected pi_e = {pi_e:.1f}%', alpha=0.7)
    if shock_type != 'None' and T > shock_period:
        axes[1].axvline(shock_period, color='red', linestyle=':', alpha=0.7, label=f'Shock at t={shock_period}')
    axes[1].set_title("Inflation (pi) Over Time")
    axes[1].set_xlabel("Time Period (t)")
    axes[1].set_ylabel("Inflation pi (%)")
    axes[1].grid(True, alpha=0.6)
    axes[1].legend(fontsize='small')
    axes[1].set_ylim(pi_lim_lower, pi_lim_upper)

    # Plot 2: Interest Rate Over Time
    axes[2].plot(time_periods, r, marker='o', linestyle='-', color='darkorange', label='Real Interest Rate r(t)', markersize=4)
    axes[2].axhline(r_bar, linestyle='--', color='gray', label=f'Natural Rate r_bar = {r_bar:.1f}%')
    if shock_type != 'None' and T > shock_period:
        axes[2].axvline(shock_period, color='red', linestyle=':', alpha=0.7, label=f'Shock at t={shock_period}')
    axes[2].set_title("Real Interest Rate (r) Over Time")
    axes[2].set_xlabel("Time Period (t)")
    axes[2].set_ylabel("Interest Rate r (%)")
    axes[2].grid(True, alpha=0.6)
    axes[2].legend(fontsize='small')
    axes[2].set_ylim(r_lim_lower, r_lim_upper)

    # Plot 3: Path in Output-Inflation Space (Phase Diagram)
    axes[3].plot(Y, pi, marker='.', linestyle='-', color='purple', label='Dynamic Path (Y(t), pi(t))')
    axes[3].scatter(Y[0], pi[0], color='red', s=100, zorder=5, label='Start (Y0, pi0)')
    axes[3].scatter(Y[-1], pi[-1], color='black', marker='X', s=100, zorder=5, label=f'End (Y{T-1}, pi{T-1})')
    # Mark the long-run equilibrium
    axes[3].scatter(Y_bar, pi_star, color='lime', marker='P', s=150, zorder=5, label='Equilibrium (Y_bar, pi*)')
    axes[3].axvline(Y_bar, linestyle='--', color='gray', alpha=0.7)
    axes[3].axhline(pi_star, linestyle='--', color='gray', alpha=0.7)
    # Add arrow to show direction of path
    if T > 1:
        axes[3].annotate('', xy=(Y[1], pi[1]), xytext=(Y[0], pi[0]),
                         arrowprops=dict(arrowstyle="->", color='purple', lw=1.5, alpha=0.6))
    if T > shock_period + 1:
         axes[3].annotate('', xy=(Y[shock_period+1], pi[shock_period+1]), xytext=(Y[shock_period], pi[shock_period]),
                         arrowprops=dict(arrowstyle="->", color='purple', lw=1.5, alpha=0.6))

    axes[3].set_title("Path in Output-Inflation Space")
    axes[3].set_xlabel("Output (Y)")
    axes[3].set_ylabel("Inflation pi (%)")
    axes[3].grid(True, alpha=0.6)
    axes[3].legend(fontsize='small')
    axes[3].set_xlim(y_lim_lower, y_lim_upper)
    axes[3].set_ylim(pi_lim_lower, pi_lim_upper)

    fig.suptitle(f'💥 AD-AS Dynamic Response to: {shock_type} Shock (Mag={shock_mag if shock_type != "None" else 0:.1f})', fontsize=16, y=1.03)
    plt.tight_layout(rect=[0, 0, 1, 0.98]) # Adjust layout
    plt.show()

    # --- Display Final Values ---
    results_md = f"""
    ### 📊 Simulation Results (at Period T={T-1})

    * **Final Output Y({T-1}):** {Y[-1]:.2f} (Potential Y_bar = {Y_bar:.1f})
    * **Final Inflation pi({T-1}):** {pi[-1]:.2f}% (Target pi* = {pi_star:.1f}%)
    * **Final Interest Rate r({T-1}):** {r[-1]:.2f}% (Natural rate r_bar = {r_bar:.1f}%)
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'} # Allow longer descriptions
interact(ad_as_shock_sim,
         pi_e=FloatSlider(value=2.0, min=0, max=5, step=0.1, description='Expected Inflation (pi_e %):', style=style),
         pi_star=FloatSlider(value=2.0, min=0, max=5, step=0.1, description='Target Inflation (pi* %):', style=style),
         Y_bar=FloatSlider(value=100, min=80, max=120, step=1, description='Potential Output (Y_bar):', style=style),
         r_bar=FloatSlider(value=2.0, min=0, max=5, step=0.1, description='Natural Rate (r_bar %):', style=style),
         phi_pi=FloatSlider(value=1.5, min=0.1, max=4.0, step=0.1, description='Taylor Rule Coeff (phi_pi):', style=style),
         lamb_as=FloatSlider(value=0.3, min=0.05, max=1.0, step=0.05, description='AS Slope (lambda):', style=style),
         alpha_is=FloatSlider(value=5.0, min=1, max=10, step=0.5, description='IS Sensitivity (alpha):', style=style),
         shock_type=Dropdown(options=['None', 'Negative Demand', 'Positive Supply', 'Negative Supply', 'Expectations'], value='None', description='Shock Type:', style=style),
         shock_mag=FloatSlider(value=5.0, min=0.0, max=10.0, step=0.5, description='Shock Size:', style=style),
         T=IntSlider(value=25, min=10, max=60, step=1, description='Periods (T):', style=style, readout_format='d'),
);


interactive(children=(FloatSlider(value=2.0, description='Expected Inflation (pi_e %):', max=5.0, style=Slider…

# 💥 AD-AS Shocks and Dynamic Adjustment

This model shows how different macroeconomic **shocks** affect the short-run:



# 🧠 Core Equations

- **Aggregate Supply (AS):**
\[
\pi_t = \pi^e + \lambda (Y_t - \bar{Y}) + \varepsilon^{AS}
\]

- **IS Curve:**
\[
Y_t = \bar{Y} - \alpha (r_t - \bar{r}) + \varepsilon^{AD}
\]

- **Monetary Policy Rule (Taylor):**
\[
r_t = \bar{r} + \phi_\pi (\pi_t - \pi^*)
\]



Each period, the economy reacts, adjusts, and returns to the steady state.

We simulate this dynamically over **T periods**.